In [1]:
# This is a sample Python script.
# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.


# See PyCharm help at https://www.jetbrains.com/help/pycharm/
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=COVID-19%20AND%20Merck%20AND%202020%202021[pdat]"
resp = requests.get(search_url).text
bs_obj = bs(resp,features="lxml")
id_list = bs_obj.find_all("id")   # idlist
print(id_list)


[<id>35441127</id>, <id>35387044</id>, <id>35373130</id>, <id>35373125</id>, <id>35373088</id>, <id>35373072</id>, <id>35291567</id>, <id>35261474</id>, <id>35178046</id>, <id>35118329</id>, <id>35118328</id>, <id>35111162</id>, <id>35095257</id>, <id>35046939</id>, <id>35039193</id>, <id>35035559</id>, <id>35035297</id>, <id>35013735</id>, <id>35007996</id>, <id>35004791</id>]


In [5]:
# print(id_list)
id_temp_list = []
for iden in id_list:
    id_temp_list.append(iden.text)
id_string = ",".join(id_temp_list)
# print(id_string)

fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id="+id_string+"&rettype=xml"
fetch_resp = requests.get(fetch_url).text
fetch_resp
fetch_resp[:2500]
fetch_obj = bs(fetch_resp,features="lxml")
for i in fetch_resp.split('<PubmedArticle>'):
    print(i)
    

<?xml version="1.0" ?>
<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2019//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_190101.dtd">
<PubmedArticleSet>
<MedlineCitation Status="PubMed-not-MEDLINE" VersionID="2" VersionDate="2022/04/04" Owner="NLM"><PMID Version="2">35441127</PMID><DateRevised><Year>2022</Year><Month>04</Month><Day>21</Day></DateRevised><Article PubModel="Electronic-eCollection"><Journal><ISSN IssnType="Electronic">2572-4754</ISSN><JournalIssue CitedMedium="Internet"><Volume>5</Volume><PubDate><Year>2021</Year></PubDate></JournalIssue><Title>Gates open research</Title><ISOAbbreviation>Gates Open Res</ISOAbbreviation></Journal><ArticleTitle>Asymptomatic COVID-19 in the elderly: dementia and viral clearance as risk factors for disease progression.</ArticleTitle><Pagination><MedlinePgn>143</MedlinePgn></Pagination><ELocationID EIdType="doi" ValidYN="Y">10.12688/gatesopenres.13357.2</ELocationID><Abstract><AbstractText Label="BACKGROUND" N

In [11]:
iden = fetch_obj.find_all("pmid")
id_list=[i.contents[0] for i in iden]
# for i in iden:
#     print(i.contents[])
#     break
# print(id_list)
titles = fetch_obj.find_all("articletitle")
title_list=[i.contents[0] for i in titles]
# print(title_list)

pub_date = fetch_obj.find_all("daterevised")
print('pub_date',pub_date)

pub_date_list=[i.contents[2].next_element+'-'+i.contents[1].next_element+"-"+i.contents[0].next_element for i in pub_date]
print(pub_date_list)

Data_dict={'PMID':id_list,'Title':title_list,'Date':pub_date_list}
# Data_dict
pub_df=pd.DataFrame.from_dict(data=Data_dict,orient='index').transpose()
print(type(fetch_obj))

pub_date [<daterevised><year>2022</year><month>04</month><day>21</day></daterevised>, <daterevised><year>2022</year><month>04</month><day>08</day></daterevised>, <daterevised><year>2022</year><month>04</month><day>07</day></daterevised>, <daterevised><year>2022</year><month>04</month><day>07</day></daterevised>, <daterevised><year>2022</year><month>04</month><day>07</day></daterevised>, <daterevised><year>2022</year><month>04</month><day>07</day></daterevised>, <daterevised><year>2022</year><month>03</month><day>18</day></daterevised>, <daterevised><year>2022</year><month>03</month><day>10</day></daterevised>, <daterevised><year>2022</year><month>02</month><day>24</day></daterevised>, <daterevised><year>2022</year><month>02</month><day>05</day></daterevised>, <daterevised><year>2022</year><month>02</month><day>05</day></daterevised>, <daterevised><year>2022</year><month>02</month><day>07</day></daterevised>, <daterevised><year>2022</year><month>02</month><day>01</day></daterevised>, <d

In [12]:
# pub_df.to_csv('./Patent.csv')
pub_df

,PMID,Title,Date
0,35441127,Asymptomatic COVID-19 in the elderly: dementia...,21-04-2022
1,35387044,Review: The Nose as a Route for Therapy. Part ...,08-04-2022
2,35373130,COVID-19 in Patients with Glomerular Disease: ...,07-04-2022
3,35373125,A Propensity Score-Matched Observational Study...,07-04-2022
4,35373088,SARS-CoV-2 Infection Risk Factors among Mainte...,07-04-2022
5,35373072,Evidence for SARS-CoV-2 Spike Protein in the U...,07-04-2022
6,35291567,The effect of the COVID-19 pandemic on influen...,18-03-2022
7,35261474,COVID-19 and the acceleration toward remote ca...,10-03-2022
8,35178046,Immune Reconstitution Following Autologous Hem...,24-02-2022
9,35118329,Systemic treatments for thymic tumors: a narra...,05-02-2022
